In [26]:
import pandas as pd
import statsmodels.api as sm
import re
import numpy as np
import pandas as pd
from itertools import product

In [27]:
def remove_pattern(col_name):
    return re.sub(r'_[0-9]{3}$', '', col_name)

Zones = pd.read_excel('Zones.xlsx')
Dataset = pd.read_csv('KAG 2016-17\Agriculture\Agriculture_KAG_2016_17.csv')
Dataset.columns = map(remove_pattern, Dataset.columns)

Working_Directory = pd.read_csv('Working_Directory.csv')
Working_Directory[24:]

,District,Rice_Production,Maize_Production,Groundnut_Production,TotalNPK,SowingSeedsDistributed_Maize,LivestockCensus_2012_Cattle,AreaUnderCereals_Maize,AreaUnderCereals_Paddy,NetAreaIrrigated_Canals_NetAreaIrrigated,AreaUnderOilSeeds_GroundNut,MotorVehicles_Tractors,LivestockCensus_2012_Sheep,NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea,Rice_Ratio,Maize_Ratio,Groundnut_Ratio,Admin_Zone
24,BALLARI,206115.5,104045,22938.5,61664.5,2859.005,172038.5,49869.5,46832.5,77607,49762,24001,756270,75198,0.269990,0.136288,0.030047,Kalaburagi Division
25,BIDAR,2036.0,2791,261.0,24658.0,24.550,234519.0,1382.0,1548.0,0,299,5256,88402,19908,0.007378,0.010114,0.000946,Kalaburagi Division
26,KALABURAGI,7888.0,14621,3114.0,53999.0,165.000,473117.0,2634.0,4199.0,20864,3817,16351,100061,48802,0.007095,0.013150,0.002801,Kalaburagi Division
27,YADGIRI,224659.0,2673,31435.0,91389.0,0.000,310727.0,961.0,49358.0,91141,38840,9902,463482,22035,0.339006,0.004033,0.047435,Kalaburagi Division
28,RAICHUR,492342.0,723,26958.0,117235.0,14.350,359124.0,2600.0,115246.0,111512,35960,25612,647549,26571,0.357505,0.000525,0.019575,Kalaburagi Division
29,KOPPAL,243505.0,190252,29982.0,73552.0,5292.000,260408.0,48210.0,50753.0,37736,22274,16317,547061,82552,0.191373,0.149521,0.023563,Kalaburagi Division
30,VIJAYANAGARA,206115.5,104045,22938.5,61664.5,2859.005,172038.5,49869.5,46832.5,77607,49762,24001,756270,75198,0.269990,0.136288,0.030047,Kalaburagi Division


In [28]:
Case_file = pd.read_csv('Params/3_parameters_multicollinearity_rice.csv')
y = Working_Directory['Rice_Production']
Case_file[:5]

,AreaUnderCereals_Paddy_190,TotalNPK_315,NetAreaIrrigated_Canals_NetAreaIrrigated_173
0,782,513.822990,0.000000
1,736,167.835355,0.000000
2,4468,257.908893,272.509463
3,1297,183.032957,9.283245
4,98259,21197.529550,18037.386430


In [29]:
if(len(Case_file) < 31):
    row_24 = Case_file.iloc[24]
    halved_row_24 = row_24 / 2
    Case_file.iloc[24] = halved_row_24
    Case_file.loc[len(Case_file)] = halved_row_24

Case_file[24:]

,AreaUnderCereals_Paddy_190,TotalNPK_315,NetAreaIrrigated_Canals_NetAreaIrrigated_173
24,46832.5,16648.820500,10476.570900
25,1548.0,181.920490,0.000000
26,4199.0,383.098388,148.020607
27,49358.0,30981.381240,30897.307860
28,115246.0,41912.073070,39866.073200
29,50753.0,14075.849460,7221.642581
30,46832.5,16648.820500,10476.570900


In [30]:
Case_file =sm.add_constant(Case_file)
Case_file.head()

,const,AreaUnderCereals_Paddy_190,TotalNPK_315,NetAreaIrrigated_Canals_NetAreaIrrigated_173
0,1.0,782.0,513.822990,0.000000
1,1.0,736.0,167.835355,0.000000
2,1.0,4468.0,257.908893,272.509463
3,1.0,1297.0,183.032957,9.283245
4,1.0,98259.0,21197.529550,18037.386430


In [31]:
model=sm.OLS(y,Case_file).fit()
print(model.summary())
model.params.values
coef=model.params.values[1:]
coef

                            OLS Regression Results                            
Dep. Variable:        Rice_Production   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.950
Method:                 Least Squares   F-statistic:                     192.7
Date:                Mon, 16 Sep 2024   Prob (F-statistic):           2.44e-18
Time:                        19:59:44   Log-Likelihood:                -363.68
No. Observations:                  31   AIC:                             735.4
Df Residuals:                      27   BIC:                             741.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

array([3.15951417, 0.98772673, 2.64630488])

In [32]:
Case_file.drop('const',axis=1,inplace=True)
Case_file[:5]

,AreaUnderCereals_Paddy_190,TotalNPK_315,NetAreaIrrigated_Canals_NetAreaIrrigated_173
0,782.0,513.822990,0.000000
1,736.0,167.835355,0.000000
2,4468.0,257.908893,272.509463
3,1297.0,183.032957,9.283245
4,98259.0,21197.529550,18037.386430


In [33]:
Case_file.loc[:,'Beta_0'] = model.params.values[0]
Case_file.loc[:,'Beta_1'] = coef[0]
Case_file.loc[:,'Beta_2'] = coef[1]
Case_file.loc[:,'Beta_3'] = coef[2]

intervals = np.arange(0, 1.1, 0.1)
columns=['Sensitivity_AreaUnderCereals_Paddy_190', 'Sensitivity_TotalNPK_315','Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173']
# Generate all possible combinations of elements in arr
combinations = product(intervals, repeat=len(columns))
combinations

In [34]:
alpha_df = pd.DataFrame(combinations, columns=columns)
alpha_df

,Sensitivity_AreaUnderCereals_Paddy_190,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173
0,0.0,0.0,0.0
1,0.0,0.0,0.1
2,0.0,0.0,0.2
3,0.0,0.0,0.3
4,0.0,0.0,0.4
...,...,...,...
1326,1.0,1.0,0.6
1327,1.0,1.0,0.7
1328,1.0,1.0,0.8
1329,1.0,1.0,0.9


In [35]:
Case_file['key'] = 1
alpha_df['key'] = 1

# Performing the Cartesian product
result_df = pd.merge( alpha_df, Case_file, on='key').drop('key', axis=1)

print(result_df)

       Sensitivity_AreaUnderCereals_Paddy_190  Sensitivity_TotalNPK_315  \
0                                         0.0                       0.0   
1                                         0.0                       0.0   
2                                         0.0                       0.0   
3                                         0.0                       0.0   
4                                         0.0                       0.0   
...                                       ...                       ...   
41256                                     1.0                       1.0   
41257                                     1.0                       1.0   
41258                                     1.0                       1.0   
41259                                     1.0                       1.0   
41260                                     1.0                       1.0   

       Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173  \
0                               

In [36]:
result_df.head()

,Sensitivity_AreaUnderCereals_Paddy_190,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173,AreaUnderCereals_Paddy_190,TotalNPK_315,NetAreaIrrigated_Canals_NetAreaIrrigated_173,Beta_0,Beta_1,Beta_2,Beta_3
0,0.0,0.0,0.0,782.0,513.822990,0.000000,-7892.626361,3.159514,0.987727,2.646305
1,0.0,0.0,0.0,736.0,167.835355,0.000000,-7892.626361,3.159514,0.987727,2.646305
2,0.0,0.0,0.0,4468.0,257.908893,272.509463,-7892.626361,3.159514,0.987727,2.646305
3,0.0,0.0,0.0,1297.0,183.032957,9.283245,-7892.626361,3.159514,0.987727,2.646305
4,0.0,0.0,0.0,98259.0,21197.529550,18037.386430,-7892.626361,3.159514,0.987727,2.646305


In [37]:
Working_Directory['Rice_Production'].mean()

131114.06451612903

In [38]:
delta_O = 5000

result_df['New_F1'] = result_df['AreaUnderCereals_Paddy_190'] + (result_df['Sensitivity_AreaUnderCereals_Paddy_190']*delta_O)/result_df['Beta_1']
result_df['New_F2'] = result_df['TotalNPK_315'] + (result_df['Sensitivity_TotalNPK_315']*delta_O)/result_df['Beta_2']
result_df['New_F3'] = result_df['NetAreaIrrigated_Canals_NetAreaIrrigated_173'] + (result_df['Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173']*delta_O)/result_df['Beta_3']
result_df['District'] = np.tile(Working_Directory['District'],11**3)

result_df['New_F1'] = result_df['New_F1'].apply(lambda x: max(0, x))
result_df['New_F2'] = result_df['New_F2'].apply(lambda x: max(0, x))
result_df['New_F3'] = result_df['New_F3'].apply(lambda x: max(0, x))

result_df

,Sensitivity_AreaUnderCereals_Paddy_190,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173,AreaUnderCereals_Paddy_190,TotalNPK_315,NetAreaIrrigated_Canals_NetAreaIrrigated_173,Beta_0,Beta_1,Beta_2,Beta_3,New_F1,New_F2,New_F3,District
0,0.0,0.0,0.0,782.0,513.822990,0.000000,-7892.626361,3.159514,0.987727,2.646305,782.000000,513.822990,0.000000,BENGALURU
1,0.0,0.0,0.0,736.0,167.835355,0.000000,-7892.626361,3.159514,0.987727,2.646305,736.000000,167.835355,0.000000,BENGALURU(R)
2,0.0,0.0,0.0,4468.0,257.908893,272.509463,-7892.626361,3.159514,0.987727,2.646305,4468.000000,257.908893,272.509463,RAMANAGARA
3,0.0,0.0,0.0,1297.0,183.032957,9.283245,-7892.626361,3.159514,0.987727,2.646305,1297.000000,183.032957,9.283245,CHITRADURGA
4,0.0,0.0,0.0,98259.0,21197.529550,18037.386430,-7892.626361,3.159514,0.987727,2.646305,98259.000000,21197.529550,18037.386430,DAVANAGERE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41256,1.0,1.0,1.0,4199.0,383.098388,148.020607,-7892.626361,3.159514,0.987727,2.646305,5781.521781,5445.227279,2037.447645,KALABURAGI
41257,1.0,1.0,1.0,49358.0,30981.381240,30897.307860,-7892.626361,3.159514,0.987727,2.646305,50940.521781,36043.510131,32786.734898,YADGIRI
41258,1.0,1.0,1.0,115246.0,41912.073070,39866.073200,-7892.626361,3.159514,0.987727,2.646305,116828.521781,46974.201961,41755.500238,RAICHUR
41259,1.0,1.0,1.0,50753.0,14075.849460,7221.642581,-7892.626361,3.159514,0.987727,2.646305,52335.521781,19137.978351,9111.069619,KOPPAL


In [39]:
result_df['New_Production'] = result_df['Beta_1'] * result_df['New_F1'] + result_df['Beta_2'] * result_df['New_F2'] + result_df['Beta_3'] * result_df['New_F3'] + result_df['Beta_0']
result_df

,Sensitivity_AreaUnderCereals_Paddy_190,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173,AreaUnderCereals_Paddy_190,TotalNPK_315,NetAreaIrrigated_Canals_NetAreaIrrigated_173,Beta_0,Beta_1,Beta_2,Beta_3,New_F1,New_F2,New_F3,District,New_Production
0,0.0,0.0,0.0,782.0,513.822990,0.000000,-7892.626361,3.159514,0.987727,2.646305,782.000000,513.822990,0.000000,BENGALURU,-4914.369577
1,0.0,0.0,0.0,736.0,167.835355,0.000000,-7892.626361,3.159514,0.987727,2.646305,736.000000,167.835355,0.000000,BENGALURU(R),-5401.448463
2,0.0,0.0,0.0,4468.0,257.908893,272.509463,-7892.626361,3.159514,0.987727,2.646305,4468.000000,257.908893,272.509463,RAMANAGARA,7199.969602
3,0.0,0.0,0.0,1297.0,183.032957,9.283245,-7892.626361,3.159514,0.987727,2.646305,1297.000000,183.032957,9.283245,CHITRADURGA,-3589.383635
4,0.0,0.0,0.0,98259.0,21197.529550,18037.386430,-7892.626361,3.159514,0.987727,2.646305,98259.000000,21197.529550,18037.386430,DAVANAGERE,371227.867246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41256,1.0,1.0,1.0,4199.0,383.098388,148.020607,-7892.626361,3.159514,0.987727,2.646305,5781.521781,5445.227279,2037.447645,KALABURAGI,21144.277832
41257,1.0,1.0,1.0,49358.0,30981.381240,30897.307860,-7892.626361,3.159514,0.987727,2.646305,50940.521781,36043.510131,32786.734898,YADGIRI,275419.509281
41258,1.0,1.0,1.0,115246.0,41912.073070,39866.073200,-7892.626361,3.159514,0.987727,2.646305,116828.521781,46974.201961,41755.500238,RAICHUR,518124.203229
41259,1.0,1.0,1.0,50753.0,14075.849460,7221.642581,-7892.626361,3.159514,0.987727,2.646305,52335.521781,19137.978351,9111.069619,KOPPAL,200475.957309


In [40]:
def normalize_list(lst):
    min_val = min(lst)
    max_val = max(lst)
    normalized_lst = [(x - min_val) / (max_val - min_val) for x in lst]
    return normalized_lst

In [41]:
import pandas as pd
import numpy as np

adjacency_excel = pd.read_excel('Karnataka_District_Adjacency_File.xlsx')

district_codes = sorted(set(adjacency_excel['District_Code']))

n = len(district_codes)
adj_matrix = np.zeros((n, n), dtype=int)

code_to_index = {code: idx for idx, code in enumerate(district_codes)}

for i in range(len(adjacency_excel)):
    district_code = adjacency_excel['District_Code'][i]
    neighbors = [int(num) for num in str(adjacency_excel['District_Neighbors'][i]).split(',') if num.strip().isdigit()]
    
    for neighbor in neighbors:
        adj_matrix[code_to_index[district_code]][code_to_index[neighbor]] = 1
        adj_matrix[code_to_index[neighbor]][code_to_index[district_code]] = 1  # Assuming adjacency is bidirectional

adj_matrix_df = pd.DataFrame(adj_matrix, index=district_codes, columns=district_codes)

print(adj_matrix_df)

adj_matrix_np = adj_matrix_df.to_numpy()

    1   2   3   4   5   6   7   8   9   10  ...  22  23  24  25  26  27  28  \
1    0   1   1   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
2    1   0   1   0   0   1   1   0   1   0  ...   0   0   0   0   0   0   0   
3    1   1   0   0   0   0   0   0   1   0  ...   0   0   0   0   0   0   0   
4    0   0   0   0   1   0   0   0   1   1  ...   0   0   0   1   0   0   0   
5    0   0   0   1   0   0   0   1   0   1  ...   0   1   0   1   0   0   0   
6    0   1   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   
7    0   1   0   0   0   1   0   0   1   0  ...   0   0   0   0   0   0   0   
8    0   0   0   0   1   0   0   0   0   1  ...   0   1   1   0   0   0   0   
9    0   1   1   1   0   0   1   0   0   1  ...   0   0   0   0   0   0   0   
10   0   0   0   1   1   0   0   1   1   0  ...   0   0   0   0   0   0   0   
11   0   0   0   0   0   0   0   0   0   1  ...   0   0   0   0   0   0   0   
12   0   0   0   0   0   0   0   1   0   1  ...   0 

In [42]:
def calculate_avg_capability(capability, neighbors):
    num_neighbors = len(neighbors)
    if num_neighbors == 0:
        return 0
    sum_capabilities = sum(capability[n] for n in neighbors)
    return sum_capabilities / num_neighbors

def calculate_stress_stability(capability, adjacency_matrix):
    n = len(capability)
    stress = np.zeros(n)
    stability = np.zeros(n)
    
    for i in range(n):
        neighbors = [j for j in range(n) if adjacency_matrix[i][j] == 1 and i != j]
        if len(neighbors) > 0:
            avg_capability = calculate_avg_capability(capability, neighbors)
            stress[i] = abs(avg_capability - capability[i])
            stability[i] = 1 - stress[i]
        else:
            stress[i] = 0  # If no neighbors, stress is zero
            stability[i] = 1
        
    return stress, stability

In [43]:
result_df['Normalized_New_Prouction'] = normalize_list(result_df['New_Production'])
result_df

,Sensitivity_AreaUnderCereals_Paddy_190,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173,AreaUnderCereals_Paddy_190,TotalNPK_315,NetAreaIrrigated_Canals_NetAreaIrrigated_173,Beta_0,Beta_1,Beta_2,Beta_3,New_F1,New_F2,New_F3,District,New_Production,Normalized_New_Prouction
0,0.0,0.0,0.0,782.0,513.822990,0.000000,-7892.626361,3.159514,0.987727,2.646305,782.000000,513.822990,0.000000,BENGALURU,-4914.369577,0.005455
1,0.0,0.0,0.0,736.0,167.835355,0.000000,-7892.626361,3.159514,0.987727,2.646305,736.000000,167.835355,0.000000,BENGALURU(R),-5401.448463,0.004529
2,0.0,0.0,0.0,4468.0,257.908893,272.509463,-7892.626361,3.159514,0.987727,2.646305,4468.000000,257.908893,272.509463,RAMANAGARA,7199.969602,0.028490
3,0.0,0.0,0.0,1297.0,183.032957,9.283245,-7892.626361,3.159514,0.987727,2.646305,1297.000000,183.032957,9.283245,CHITRADURGA,-3589.383635,0.007975
4,0.0,0.0,0.0,98259.0,21197.529550,18037.386430,-7892.626361,3.159514,0.987727,2.646305,98259.000000,21197.529550,18037.386430,DAVANAGERE,371227.867246,0.720680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41256,1.0,1.0,1.0,4199.0,383.098388,148.020607,-7892.626361,3.159514,0.987727,2.646305,5781.521781,5445.227279,2037.447645,KALABURAGI,21144.277832,0.055005
41257,1.0,1.0,1.0,49358.0,30981.381240,30897.307860,-7892.626361,3.159514,0.987727,2.646305,50940.521781,36043.510131,32786.734898,YADGIRI,275419.509281,0.538503
41258,1.0,1.0,1.0,115246.0,41912.073070,39866.073200,-7892.626361,3.159514,0.987727,2.646305,116828.521781,46974.201961,41755.500238,RAICHUR,518124.203229,1.000000
41259,1.0,1.0,1.0,50753.0,14075.849460,7221.642581,-7892.626361,3.159514,0.987727,2.646305,52335.521781,19137.978351,9111.069619,KOPPAL,200475.957309,0.396000


In [44]:
stress_values_list = []
stability_values_list = []

# Define batch size
batch_size = 31

# Iterate over the New_Production column in batches of 31
num_batches = len(result_df) // batch_size

for i in range(num_batches):
    # Get the batch of 31 production values
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch = normalize_list(result_df['New_Production'][start_idx:end_idx].values)
    
    # Calculate the stress and stability values for the batch
    stress_batch, stability_batch = calculate_stress_stability(batch, adj_matrix_np)
    
    stress_values_list.extend(stress_batch)
    stability_values_list.extend(stability_batch)


print(num_batches)
print(len(stress_batch))
print(len(stress_values_list), len(stability_values_list))

result_df['Stress_Values'] = stress_values_list
result_df['Stability_Values'] = stability_values_list

1331
31
41261 41261


In [45]:
result_df.columns

Index(['Sensitivity_AreaUnderCereals_Paddy_190', 'Sensitivity_TotalNPK_315',
       'Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173',
       'AreaUnderCereals_Paddy_190', 'TotalNPK_315',
       'NetAreaIrrigated_Canals_NetAreaIrrigated_173', 'Beta_0', 'Beta_1',
       'Beta_2', 'Beta_3', 'New_F1', 'New_F2', 'New_F3', 'District',
       'New_Production', 'Normalized_New_Prouction', 'Stress_Values',
       'Stability_Values'],
      dtype='object')

In [46]:
viz_df = result_df[['District','Sensitivity_AreaUnderCereals_Paddy_190','Sensitivity_TotalNPK_315','Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173',
                    'New_Production','Stability_Values','Normalized_New_Prouction']].copy()

abbreviation_mapping = {
    'BENGALURU': 'BLR',
    'BENGALURU(R)': 'BLR(R)',
    'RAMANAGARA': 'RGA',
    'CHITRADURGA': 'CDA',
    'DAVANAGERE': 'DVG',
    'KOLAR': 'KLR',
    'CHIKKABALLAPURA': 'CKA',
    'SHIVAMOGGA': 'SMG',
    'TUMAKURU': 'TKR',
    'CHIKKAMAGALURU': 'CMG',
    'DAKSHINA KANNADA': 'DKA',
    'UDUPI': 'UPI',
    'HASSAN': 'HSN',
    'KODAGU': 'KDG',
    'MANDYA': 'MDY',
    'MYSURU': 'MYS',
    'CHAMARAJANAGAR': 'CNR',
    'BELAGAVI': 'BLG',
    'VIJAYAPURA': 'VJP',
    'BAGALKOT': 'BKT',
    'DHARAWAD': 'DWD',
    'GADAG': 'GDG',
    'HAVERI': 'HVR',
    'UTTARA KANNADA': 'UTK',
    'BALLARI': 'BLL',
    'BIDAR': 'BDR',
    'KALABURAGI': 'KLB',
    'YADGIRI': 'YDR',
    'RAICHUR': 'RCR',
    'KOPPAL': 'KPL',
    'Vijayanagara' : 'VJN'
}

viz_df['Abbreviation'] = viz_df['District'].map(abbreviation_mapping)

legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                    for abb, full_form in zip(viz_df['Abbreviation'].unique(), viz_df['District'].unique())]

In [47]:
viz_df['New_Production'] = viz_df['New_Production'].apply(lambda x: max(0, x))

viz_df['Percentage Change'] = viz_df['New_Production'] - np.tile(y.values, len(viz_df) // 31 + 1)[:len(viz_df)]
for i in range(0, len(viz_df), 31):
    viz_df.loc[i:i+30, 'Percentage Change'] = viz_df.loc[i:i+30, 'Percentage Change'] * 100 / y.values + 1e-9
    viz_df.loc[i:i+30, 'Normalized Percentage Change'] = normalize_list(viz_df.loc[i:i+30, 'Percentage Change'])


viz_df['Percentage Change'] = viz_df['Percentage Change'].replace([np.inf, -np.inf], 0)

# viz_df['Normalized Percentage Change'] = normalize_list(viz_df['Percentage Change'])


In [48]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output
import plotly.graph_objects as go

def update_plot(alpha1, alpha2, alpha3):
    
    tolerance = 1e-5 
    filtered_df = viz_df[
        (viz_df['Sensitivity_AreaUnderCereals_Paddy_190'].between(alpha1 - tolerance, alpha1 + tolerance)) &
        (viz_df['Sensitivity_TotalNPK_315'].between(alpha2 - tolerance, alpha2 + tolerance)) &
        (viz_df['Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173'].between(alpha3 - tolerance, alpha3 + tolerance)) 
    ]

    # Create a scatter plot with filtered data
    fig = px.scatter(filtered_df, x='Stability_Values', y='Normalized Percentage Change', title='Rice Impact vs Stability',
                     labels={'Normalized Percentage Change': 'Impact', 'Stability_Values': 'Stability'},
                     text='Abbreviation', opacity=1, color_discrete_sequence=['darkblue'])

    fig.update_traces(textposition='top center', textfont=dict(size=11))

    # Update layout to add annotations and legends
    fig.update_layout(
        annotations=[
            dict(
                x=0.5,
                y=-0.25,
                showarrow=False,
                text=f'α<sub>AP</sub>: {alpha1:.2f}, α<sub>NPK</sub>: {alpha2:.2f}, α<sub>CN</sub>: {alpha3:.2f}',
                xref="paper",
                yref="paper",
                font=dict(size=13)
            )
        ],
        legend_title_text='Districts',
        legend_title=dict(font=dict(size=12)),
    )

    # Adding custom legends with abbreviations and full forms
    legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                     for abb, full_form in zip(filtered_df['Abbreviation'].unique(), filtered_df['District'].unique())]

    for label in legend_labels:
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(color='darkblue'),
            name=f"{label['Abbreviation']}: {label['Full Form']}"
        ))

    # Adding average lines for stability and percentage change
    avg_stability = filtered_df['Stability_Values'].mean()
    avg_precent = filtered_df['Normalized Percentage Change'].mean()

    fig.add_shape(
        type='line',
        x0=avg_stability, y0=0, x1=avg_stability, y1=1,
        line=dict(color='red', width=1.5, dash='dash'),
        yref='paper'
    )

    fig.add_shape(
        type='line',
        x0=0, y0=avg_precent, x1=1, y1=avg_precent,
        line=dict(color='red', width=1.5, dash='dash'),
        xref='paper'
    )

    # Add annotations for average values
    fig.add_trace(go.Scatter(
        x=[avg_stability],
        y=[-0.07],
        text=[f'Avg Stability: {avg_stability:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    fig.add_trace(go.Scatter(
        x=[0.3],
        y=[avg_precent-0.04],
        text=[f'Avg Impact: {avg_precent:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    clear_output()

    display(fig)

# Creating sliders for interaction
alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αPM:')
alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αNPK:')
alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αCN:')

# Creating interactive plot
interactive_plot = interact(update_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider)

# Create update button to trigger plot update
update_button = widgets.Button(description="Update Plot")
update_button.on_click(lambda _: update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value))

# Display the button and sliders
widgets.HBox([update_button, alpha1_slider, alpha2_slider, alpha3_slider])


interactive(children=(FloatSlider(value=0.5, description='αPM:', max=1.0), FloatSlider(value=0.5, description=…

In [49]:
result_df.to_csv('Rice_Prescriptive_5000.csv',index=False)